# RAG: Data Ingestion and Retrieval for complex documents



In [2]:
import time
from typing import List, Optional, Union

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from rag_101.retriever import (
    create_parent_retriever,
    load_embedding_model,
    load_pdf,
    load_reranker_model,
    retrieve_context,
)
from rich import print

Load PDF documents and create retriever

In [3]:
files = [
    "/teamspace/studios/this_studio/example_data/x-t30.pdf",  # DocLLM paper
]

docs = load_pdf(files=files)

embedding_model = load_embedding_model()
retriever = create_parent_retriever(docs, embedding_model)
reranker_model = load_reranker_model()

The PDF <_io.BufferedReader name='/teamspace/studios/this_studio/example_data/x-t30.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Initialize Langchain's Ollama Chat API integration using `mistral` model and create the chain with prompt template.

In [4]:
llm = ChatOllama(model="mistral")
prompt_template = ChatPromptTemplate.from_template(
    (
        "Please answer the following question based on the provided `context` that follows the question.\n"
        "If you do not know the answer then just say 'I do not know'\n"
        "question: {question}\n"
        "context: ```{context}```\n"
    )
)
chain = prompt_template | llm | StrOutputParser()

Retrieve document and run the chain using `context` and `question`.

In [5]:
query = "What is the source of the dataset the model was trained on?"
context, similarity_score = retrieve_context(query, retriever, reranker_model)[0]
context = context.page_content

output = chain.invoke({"context": context, "question": query})
print("LLM Response:", output)

LLM Response:  I do not know. The context provided does not give any information about the source of the dataset 
the model was trained on.

In [13]:
# Run through some sample queries
samples = [
    "How do I configure my camera to take photos automatically at a preset interval",
    "How do I adjust camera's sensitivity to light?",
    "My pictures are turning out darker than I want, what should I do"
]

In [14]:
for query in samples:
    print(query)
    context = retrieve_context(
        query, retriever=retriever, reranker_model=reranker_model
    )[0]
    output = chain.invoke({"context": context[0].page_content, "question": query})
    print("LLM Response:", output)

    print("\n", "=" * 100, "\n\n")

How do I configure my camera to take photos automatically at a preset interval

LLM Response:  To configure your camera to take photos automatically at a preset interval, follow these steps as 
described in the context:

1. Highlight INTERVAL TIMER SHOOTING in the A (SHOOTING SETTING) tab and press MENU/OK.
2. Use the focus stick (focus lever) to choose the interval and number of shots. Press MENU/OK to proceed.
3. Use the focus stick to choose the starting time and then press MENU/OK. Shooting will start automatically.

Remember that interval timer photography cannot be used at a shutter speed of B (bulb) or with multiple exposure 
photography. Using a tripod is recommended for stability, and an AC-5VG AC power adapter can be used to ensure a 
continuous power supply for longer shoots. The display turns off between shots but can be activated by pressing the
shutter button at any time. To continue shooting until the memory card is full, set the number of shots to ∞.

====================================================================================================

How do I adjust camera's sensitivity to light?

LLM Response:  To adjust the camera's sensitivity to light, look for the ISO setting in the shooting menus. You can
find it on line 13 and 14 of the provided context. There are several options for adjusting sensitivity: AUTO1, 
AUTO2, AUTO3, H (51200), H (25600), L (125), and L (100). The camera can automatically adjust sensitivity in 
response to shooting conditions using the AUTO options, or you can manually select a specific value. Keep in mind 
that some restrictions may apply when using certain features like the electronic shutter or image stabilization 
with motion detection.

====================================================================================================

My pictures are turning out darker than I want, what should I do

LLM Response:  Based on the context provided, if your pictures are turning out darker than you want, you may want 
to adjust the Exposure Compensation setting. Unfortunately, this option is not shown in the provided context. 
However, you can try adjusting the Peripheral Illumination Correction or Color Shading Correction settings as they 
might affect the brightness of your images to some extent. To access these settings, follow the steps in the 
context for "Color Shading Correction" and "Peripheral Illumination Correction". Additionally, checking the 
exposure settings (Shutter Speed, Aperture, and ISO) can also help improve the image's brightness.

====================================================================================================